In [5]:
!wget https://www.machinehack.com/wp-content/uploads/2019/09/Participants_Data-20190926T094854Z-001.zip

--2019-10-03 13:03:09--  https://www.machinehack.com/wp-content/uploads/2019/09/Participants_Data-20190926T094854Z-001.zip
Resolving www.machinehack.com (www.machinehack.com)... 107.180.71.20
Connecting to www.machinehack.com (www.machinehack.com)|107.180.71.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3521673 (3.4M) [application/zip]
Saving to: ‘Participants_Data-20190926T094854Z-001.zip’

Participants_Data-2 100%[===================>]   3.36M  1.24MB/s    in 2.7s    

2019-10-03 13:03:13 (1.24 MB/s) - ‘Participants_Data-20190926T094854Z-001.zip’ saved [3521673/3521673]



In [6]:
!unzip /content/Participants_Data-20190926T094854Z-001.zip

Archive:  /content/Participants_Data-20190926T094854Z-001.zip
  inflating: Participants_Data/Data_Test.xlsx  
  inflating: Participants_Data/Data_Train.xlsx  
  inflating: Participants_Data/Sample_Submission.xlsx  


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
import gc
%matplotlib inline

In [0]:
train = pd.read_excel('Participants_Data/Data_Train.xlsx')
test = pd.read_excel('Participants_Data/Data_Test.xlsx')
sub = pd.read_excel('Participants_Data/Sample_Submission.xlsx')

In [142]:
train.head(2)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,Ratings_int,Reviews_int,ratio_of_review,Released_year,Reviews_Ratings
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,8,4.0,0.80,2016,32.0
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,14,3.9,0.78,2012,54.6


In [0]:
for df in [train,test]:
  df.pop('Reviews')
  df.pop('Ratings')

In [0]:
for df in [train,test]:
  df['Ratings_int'] = df['Ratings_int'].map(lambda x:x.split(','))
  df['Ratings_int'] = df['Ratings_int'].map(lambda x:''.join(x))
  df['Ratings_int'] = df['Ratings_int'].astype(int)
  df['Reviews_Ratings'] = df['Reviews_int']*df['Ratings_int']

In [0]:
for df in [train,test]:
  df['Released_year'] = df['Edition'].map(lambda x:x.split(' ')[-1])

In [0]:
for df in [train,test]:
  df['Synopsis'] = TfidfVectorizer().fit_transform(df['Synopsis'])

In [0]:
for df in [train,test]:
  df['Reviews_int'] = df['Reviews'].map(lambda x:x.split(' ')[0])

In [0]:
for df in [train,test]:
  df['Reviews_int'] = df['Reviews_int'].astype(float)
  df['ratio_of_review'] = df['Reviews_int']/5

In [0]:
for df in [train,test]:
  df['Ratings_int'] = df['Ratings'].map(lambda x:x.split(' ')[0])

In [0]:
le = LabelEncoder()

In [0]:
for c in test.columns:
  for df in [train,test]:
    if df[c].dtype == 'object':
      df[c] = le.fit_transform(df[c])

In [0]:
x = train.drop('Price',axis=1)
y = train['Price']

In [0]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.01)

In [0]:
folds = KFold(n_splits=5)

lgb_fold_pred = 0
lgb_val_pred = 0

lr = 0.835
step = 0.6985

#validation_data = lgb.Dataset(validation_x,label=validation_y)

for fold, (trn_idx, test_idx) in enumerate(folds.split(x_train)):
    print('========================================================================')
    print('Training on fold {}'.format(fold + 1))
    
    trn_data = lgb.Dataset(x_train.iloc[trn_idx], label=y_train.iloc[trn_idx])
    
    val_data = lgb.Dataset(x_train.iloc[test_idx], label=y_train.iloc[test_idx])
    
    params = {
            'min_data_in_leaf': 280, 
            'num_leaves': 430, 
            'learning_rate': lr,
            'min_child_weight': 0.0227696,
            'bagging_fraction': 0.625789, 
            'feature_fraction': 0.47,
            'reg_lambda': 0.72,
            'reg_alpha': 0.46,
            'max_depth': -1, 
            'objective': 'regression',
            'seed': 59,
            'tree_learner':'serial',
            'feature_fraction_seed': 42,
            'n_jobs':-1,
            'bagging_seed': 42,
            'drop_seed': 49,
            'data_random_seed': 42,
            'boosting_type': 'goss',
            'verbose': 1,
            'metric':'rmse',
            'od_wait':500,
            }
        
    clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds=50)
    
    lgb_fold_pred += clf.predict(test)
    lgb_val_pred += clf.predict(x_test)
    
    lr = lr * step**((fold/5))
    step = -1*step*np.log(0.8)
    
    del trn_data,val_data
    gc.collect()
    
lgb_val_pred /= 5
lgb_fold_pred /= 5

In [176]:
lgb_val_pred

array([505.71042438, 424.58776232, 599.7508568 , 350.17152488,
       439.93935817, 471.38513627, 772.15070301, 426.04908698,
       488.28402158, 430.6081608 , 693.74020701, 484.44426437,
       743.17184835, 570.9561335 , 469.42347335, 552.72952224,
       610.28776849, 576.22799353, 482.39062387, 551.13190586,
       422.13507696, 636.02349483, 702.13489604, 406.6469155 ,
       440.84303513, 904.72217525, 423.19155114, 417.65489838,
       336.87816956, 560.55657203, 455.80440219, 646.94500758,
       466.558947  , 499.63942832, 531.94677759, 644.09588777,
       607.42435126, 438.80459107, 445.55880524, 558.47870557,
       482.02252217, 724.47929156, 735.5620427 , 581.50298668,
       594.88920627, 670.03407031, 579.47669796, 435.13669546,
       393.00395024, 456.01050621, 415.55505847, 567.48939739,
       453.49336163, 661.39081245, 660.35060109, 416.63853811,
       783.84264276, 641.22347201, 454.14804868, 724.13336922,
       418.58028973, 472.83679355, 423.07864558])

In [0]:
sub['Price'] = lgb_fold_pred
sub.to_excel('book_prediction.xlsx',index=False)